In [8]:
#Finalizing Deliverables: Audit, BI Export, and Executive Reporting
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import datetime

# --- 1. RELOAD & PREDICT (Restoring the 'df' and 'accuracy_pct') ---
df = pd.read_csv('loan_data_cleaned.csv')
df.columns = df.columns.str.strip()
df['loan_status_encoded'] = df['loan_status'].str.strip().map({'Approved': 1, 'Rejected': 0})

# Load the saved AI brain
model = joblib.load(r'C:\Users\91950\loan_prediction_model.pkl')
model_columns = joblib.load(r'C:\Users\91950\model_columns.pkl')

# Align data and predict
X = df.drop(['loan_id', 'loan_status', 'loan_status_encoded'], axis=1, errors='ignore')
X = pd.get_dummies(X, drop_first=True)
X_aligned = X.reindex(columns=model_columns, fill_value=0)
df['AI_Prediction'] = model.predict(X_aligned)
df['Model_Accuracy'] = df['loan_status_encoded'] == df['AI_Prediction']
accuracy_pct = df['Model_Accuracy'].mean() * 100

# --- STEP 1: AUDIT REPORT ---
disagreements = df[df['Model_Accuracy'] == False].copy()
def categorize_error(row):
    return 'AI Too Strict' if row['loan_status_encoded'] == 1 else 'AI Too Lenient'
disagreements['Error_Type'] = disagreements.apply(categorize_error, axis=1)
disagreements.to_csv('loan_audit_review.csv', index=False)

# --- STEP 2: POWER BI EXPORT ---
master_df = df.copy()
master_df['AI_Result'] = master_df['AI_Prediction'].map({1: 'Approved', 0: 'Rejected'})
master_df['Accuracy_Status'] = master_df['Model_Accuracy'].map({True: 'Correct', False: 'Disagreement'})
master_df.to_csv('loan_data_for_powerbi_final.csv', index=False)

# --- STEP 3: PDF EXECUTIVE SUMMARY ---
# Get feature importance for the report
importances = pd.Series(model.feature_importances_, index=model_columns).sort_values(ascending=False)

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", 'B', 18)
pdf.cell(200, 15, txt="Loan Risk AI: Executive Summary", ln=True, align='C')
pdf.set_font("Arial", size=11)
pdf.cell(200, 10, txt=f"Analysis Date: {datetime.date.today()}", ln=True, align='C')
pdf.ln(10)

pdf.set_font("Arial", 'B', 12)
pdf.cell(0, 10, "1. Performance Summary", ln=True)
pdf.set_font("Arial", size=10)
pdf.multi_cell(0, 8, f"- Prediction Accuracy: {accuracy_pct:.2f}%\n"
                      f"- Primary Driver: {importances.index[0].upper()}\n"
                      f"- Total Records Audited: {len(df)}")
pdf.ln(5)

pdf.set_font("Arial", 'B', 12)
pdf.cell(0, 10, "2. Strategic Recommendations", ln=True)
pdf.set_font("Arial", size=10)
pdf.multi_cell(0, 8, "1. Deploy AI for instant low-risk approvals.\n"
                      "2. Use the 'Disagreement' audit for senior manager review.\n"
                      "3. Integrate Power BI dashboard for real-time monitoring.")

pdf.output("Loan_Analysis_Report.pdf")

print(" SUCCESS! 3 Files have been created in your folder:")
print("1. loan_audit_review.csv (Step 1)")
print("2. loan_data_for_powerbi_final.csv (Step 2)")
print("3. Loan_Analysis_Report.pdf (Step 3)")

 SUCCESS! 3 Files have been created in your folder:
1. loan_audit_review.csv (Step 1)
2. loan_data_for_powerbi_final.csv (Step 2)
3. Loan_Analysis_Report.pdf (Step 3)
